In [104]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

#Data processing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, accuracy_score, confusion_matrix

from xgboost import XGBClassifier

['weatherAUS.csv']


In [79]:
data_full = pd.read_csv("../input/weatherAUS.csv")

In [80]:
print(data_full.columns) # Get the column names
print(data_full.dtypes)

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RISK_MM', 'RainTomorrow'],
      dtype='object')
Date              object
Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainToday         object
RISK_MM          float64
RainTomorrow      object
dtype: object


In [81]:
data_full.RainTomorrow.isna().value_counts() # Make sure all our target data is present

False    142193
Name: RainTomorrow, dtype: int64

In [82]:
X_full = data_full.drop(["Date", "RISK_MM", "RainTomorrow"], axis = 1) # drop target and data leak (RISK_MM)
y_full = data_full.RainTomorrow # target

In [83]:
X = pd.get_dummies(X_full)

In [84]:
imputer = Imputer(strategy="most_frequent")
X = pd.DataFrame(imputer.fit_transform(X), columns = X.columns)

In [85]:
X_train, x_valid, y_train, y_valid = train_test_split(X, y_full, random_state = 1, test_size=0.1)

In [88]:
clf = XGBClassifier(n_estimators=500)
clf.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(x_valid, y_valid)])

[0]	validation_0-error:0.162236
Will train until validation_0-error hasn't improved in 5 rounds.
[1]	validation_0-error:0.162799
[2]	validation_0-error:0.160056
[3]	validation_0-error:0.160127
[4]	validation_0-error:0.159142
[5]	validation_0-error:0.159564
[6]	validation_0-error:0.158579
[7]	validation_0-error:0.159142
[8]	validation_0-error:0.157314
[9]	validation_0-error:0.15661
[10]	validation_0-error:0.155978
[11]	validation_0-error:0.155134
[12]	validation_0-error:0.155134
[13]	validation_0-error:0.155837
[14]	validation_0-error:0.154501
[15]	validation_0-error:0.15443
[16]	validation_0-error:0.15429
[17]	validation_0-error:0.152391
[18]	validation_0-error:0.152321
[19]	validation_0-error:0.151406
[20]	validation_0-error:0.151547
[21]	validation_0-error:0.151758
[22]	validation_0-error:0.151266
[23]	validation_0-error:0.151266
[24]	validation_0-error:0.150703
[25]	validation_0-error:0.150703
[26]	validation_0-error:0.150492
[27]	validation_0-error:0.15
[28]	validation_0-error:0.14

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=500,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [89]:
y_pred = clf.predict(x_valid)

In [107]:
accuracy_score(y_valid, y_pred)

0.8504219409282701

In [108]:
confusion_matrix(y_valid, y_pred)

array([[10675,   469],
       [ 1658,  1418]])